<a href="https://colab.research.google.com/github/lenust/neuronet_holes_in_leaves/blob/main/To_train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connect your Google Drive and specify the folder where your data are located. This folder should contain the "leaves_for_modelTRAIN" folder, which contains intact leaves for model training

In [1]:
from google.colab import drive
drive.mount('/content/drive')
google_drive_folder = "/content/drive/MyDrive/neuronet for damaged area calculation" # Replace with the path to your folder if necessary

Mounted at /content/drive


Create functions to generate artificial damage on intact leaves. First method: Randomly generating groups of bites with each group having a random center point and radius, resulting in multiple concentric circles within that radius.

In [3]:
from PIL import Image, ImageDraw
import os
import random

# Define a function to to generate artificial damage (method1)
def add_random_circles_to_image(image, min_num_groups, max_num_groups, min_radius, max_radius, min_circles, max_circles, min_circle_radius, max_circle_radius):
    # Create a drawing object on the image
    draw = ImageDraw.Draw(image)

    # Generate a random number of groups of circles
    num_groups = random.randint(min_num_groups, max_num_groups)
    for i in range(num_groups):
        # Choose a random point within the image
        x = random.randint(0, image.width)
        y = random.randint(0, image.height)

        # Choose a random radius for this group of circles
        radius = random.randint(min_radius, max_radius)

        # Generate a random number of circles within this radius
        num_circles = random.randint(min_circles, max_circles)
        for j in range(num_circles):
            # Generate random coordinates for each circle within the group
            point_x = random.randint(x - radius, x + radius)
            point_y = random.randint(y - radius, y + radius)

            # Choose a random radius for each circle within the group
            circle_radius = random.randint(min_circle_radius, int(radius * max_circle_radius))

            # Draw the circle
            draw.ellipse((point_x - circle_radius, point_y - circle_radius, point_x + circle_radius, point_y + circle_radius), fill=(255, 255, 255))

    return image

# Define a function to  to generate artificial damage (method1) to all images in a folder
def add_random_circles_to_folder(input_folder, output_folder, min_num_groups, max_num_groups, min_radius, max_radius, min_circles, max_circles, min_circle_radius, max_circle_radius):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate through all files in the input folder
    for filename in os.listdir(input_folder):
        # Ignore files that are not image files
        if not filename.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
            continue

        # Open the image
        with Image.open(os.path.join(input_folder, filename)) as image:
            # Add random circles to the image
            image_with_circles = add_random_circles_to_image(image, min_num_groups, max_num_groups, min_radius, max_radius, min_circles, max_circles, min_circle_radius, max_circle_radius)

            # Save the modified image to the output folder
            new_filename = os.path.join(output_folder, filename)
            image_with_circles.save(new_filename)

Second method: Randomly generating groups of bites, each with a random center point and a circle with a random radius. Subsequently, a random number of points were selected along the circumference of the initial circle, and additional circles with random radii were drawn at those points.

In [4]:
# Import necessary libraries and modules
from PIL import Image, ImageDraw
import os
import random
import math

# Define a function to to generate artificial damage (method2)
def add_random_circles_to_image2(image, min_num_groups, max_num_groups, min_radius, max_radius, min_circles, max_circles):
    # Create a drawing object on the image
    draw = ImageDraw.Draw(image)

    # Generate a random number of groups of circles
    num_groups = random.randint(min_num_groups, max_num_groups)
    for i in range(num_groups):
        # Choose a random point within the image
        x = random.randint(0, image.width)
        y = random.randint(0, image.height)

        # Choose a random radius for this group of circles and draw the enclosing ellipse
        radius = random.randint(min_radius, max_radius)
        draw.ellipse((x - radius, y - radius, x + radius, y + radius), fill=(255, 255, 255))

        # Generate a random number of circles within this radius
        num_circles = random.randint(min_circles, max_circles)
        for i in range(num_circles):
            # Choose a random angle for the point
            angle = random.uniform(0, 2 * math.pi)
            # Calculate the coordinates of the point on the radius
            point_x = x + int(radius * math.cos(angle))
            point_y = y + int(radius * math.sin(angle))
            # Choose a random radius for the circle and draw it
            circle_radius = random.randint(min_radius, max_radius)
            draw.ellipse((point_x - circle_radius, point_y - circle_radius, point_x + circle_radius, point_y + circle_radius), fill=(255, 255, 255))

    return image

# Define a function to generate artificial damage (method2) to all images in a folder
def add_random_circles_to_folder2(input_folder, output_folder, min_num_groups, max_num_groups, min_radius, max_radius, min_circles, max_circles):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate through all files in the input folder
    for filename in os.listdir(input_folder):
        # Ignore files that are not image files
        if not filename.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
            continue

        # Open the image
        with Image.open(os.path.join(input_folder, filename)) as image:
            # Add random circles to the image using the previous function
            image_with_circles = add_random_circles_to_image2(image, min_num_groups, max_num_groups, min_radius, max_radius, min_circles, max_circles)

            # Save the modified image to the output folder
            new_filename = os.path.join(output_folder, filename)
            image_with_circles.save(new_filename)



Create the folder structure required for the model and call the functions to generate artificial damage on intact leaves with the specified parameters to create damages of high, medium and low degrees.

In [ ]:
import os
import shutil

# Create the 'leaves_for_modelTRAIN' folder and move images into 'source_leaves'
leaves_for_modelTRAIN_folder = os.path.join(google_drive_folder, 'leaves_for_modelTRAIN')
source_leaves_folder = os.path.join(leaves_for_modelTRAIN_folder, 'source_leaves')

# Create the 'source_leaves' folder if it doesn't exist
if not os.path.exists(source_leaves_folder):
    os.makedirs(source_leaves_folder)

# Move images from 'leaves_for_modelTRAIN' to 'source_leaves'
for filename in os.listdir(leaves_for_modelTRAIN_folder):
    # Check if the file is an image (e.g., .jpg, .jpeg, .png, etc.)
    if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
        # Get the full path to the current image
        image_path = os.path.join(leaves_for_modelTRAIN_folder, filename)

        # Move the image to the 'source_leaves' folder
        shutil.move(image_path, os.path.join(source_leaves_folder, filename))

# Create 'Without' and 'With' folders
without_folder = os.path.join(leaves_for_modelTRAIN_folder, 'Without')
with_folder = os.path.join(leaves_for_modelTRAIN_folder, 'With')

# Create 'Without' and 'With' folders if they don't exist
if not os.path.exists(without_folder):
    os.makedirs(without_folder)
if not os.path.exists(with_folder):
    os.makedirs(with_folder)

# Create 7 copies of the 'source_leaves' folder inside 'Without'
copy_folders = ['with_medium', 'with_small', 'with_big', 'with_medium2', 'with_small2', 'with_big2', 'without']
for folder_name in copy_folders:
    folder_path = os.path.join(without_folder, folder_name)
    # Create a copy of the 'source_leaves' folder
    shutil.copytree(source_leaves_folder, folder_path)

# Create 1 copy of the 'source_leaves' folder inside 'With'
with_without_folder = os.path.join(with_folder, 'without')
# Create a copy of the 'source_leaves' folder
shutil.copytree(source_leaves_folder, with_without_folder)

# Get paths to folders for use in the 'add_random_circles_to_folder' function
input_folder = source_leaves_folder
with_medium_folder = os.path.join(with_folder, 'with_medium')
with_small_folder = os.path.join(with_folder, 'with_small')
with_big_folder = os.path.join(with_folder, 'with_big')
with_medium_folder2 = os.path.join(with_folder, 'with_medium2')
with_small_folder2 = os.path.join(with_folder, 'with_small2')
with_big_folder2 = os.path.join(with_folder, 'with_big2')

# Create the folders if they don't exist
for folder in [with_medium_folder, with_small_folder, with_big_folder, with_medium_folder2, with_small_folder2, with_big_folder2]:
    if not os.path.exists(folder):
        os.makedirs(folder)

# Call the 'add_random_circles_to_folder' and 'add_random_circles_to_folder2' functions with the specified parameters
add_random_circles_to_folder(input_folder, with_medium_folder, min_num_groups=2, max_num_groups=3, min_radius=10, max_radius=60, min_circles=3, max_circles=6, min_circle_radius=5, max_circle_radius=2/3)
add_random_circles_to_folder(input_folder, with_small_folder, min_num_groups=2, max_num_groups=3, min_radius=10, max_radius=20, min_circles=2, max_circles=5, min_circle_radius=2, max_circle_radius=3/4)
add_random_circles_to_folder(input_folder, with_big_folder, min_num_groups=3, max_num_groups=6, min_radius=10, max_radius=70, min_circles=4, max_circles=8, min_circle_radius=5, max_circle_radius=2/3)

add_random_circles_to_folder2(input_folder, with_small_folder2, min_num_groups=1, max_num_groups=3, min_radius=1, max_radius=10, min_circles=1, max_circles=10)
add_random_circles_to_folder2(input_folder, with_medium_folder2, min_num_groups=2, max_num_groups=4, min_radius=3, max_radius=15, min_circles=3, max_circles=12)
add_random_circles_to_folder2(input_folder, with_big_folder2, min_num_groups=5, max_num_groups=10, min_radius=5, max_radius=20, min_circles=5, max_circles=20)


Create a folder with images for testing the model results

In [6]:
# Define the path to the 'test' folder
test_folder = os.path.join(leaves_for_modelTRAIN_folder, 'test')

# Create the 'test' folder if it doesn't exist
if not os.path.exists(test_folder):
    os.makedirs(test_folder)

# Define the path to the 'test_with' folder inside 'test'
test_with_folder = os.path.join(test_folder, 'test_with')

# Create the 'test_with' folder inside 'test' if it doesn't exist
if not os.path.exists(test_with_folder):
    os.makedirs(test_with_folder)

# List of folders from which to take two files each
source_folders = [with_medium_folder, with_small_folder, with_big_folder, with_medium_folder2, with_small_folder2, with_big_folder2]

# Loop through each of the specified folders
for source_folder in source_folders:
    # Get the list of files in the current folder
    files = os.listdir(source_folder)

    # Take the first two files (if they exist)
    for i in range(2):
        if i < len(files):
            file_name = files[i]
            # Full path to the file in the source folder
            source_file_path = os.path.join(source_folder, file_name)
            # Full path to the target folder ('test_with')
            target_file_path = os.path.join(test_with_folder, file_name)
            # Move the file to 'test_with'
            shutil.move(source_file_path, target_file_path)

# Define the path to the 'test_without' folder inside 'test'
test_without_folder = os.path.join(test_folder, 'test_without')

# Create the 'test_without' folder inside 'test' if it doesn't exist
if not os.path.exists(test_without_folder):
    os.makedirs(test_without_folder)

# Get the list of files in the 'with_without_folder'
without_files = os.listdir(with_without_folder)

# Take the first two files (if they exist)
for i in range(2):
    if i < len(without_files):
        file_name = without_files[i]
        # Full path to the file in the source 'without' folder
        source_file_path = os.path.join(with_without_folder, file_name)

        # Create five copies of the file
        for j in range(5):
            # Full path to the target folder 'test_without'
            target_file_name = f"{file_name.split('.')[0]}_{j}.{file_name.split('.')[-1]}"
            target_file_path = os.path.join(test_without_folder, target_file_name)

            # Copy the file to 'test_without'
            shutil.copy(source_file_path, target_file_path)


Create objects of dataset class

In [7]:
import os
import torch
from PIL import Image
import numpy as np
import cv2
import torchvision.transforms as transforms
import torchvision.datasets as datasets

# Define a function to load an image from a given path
def load_image(path):
    with open(path, 'rb') as f:
        img = Image.open(f)
        return transforms.Compose([transforms.ToTensor(), transforms.Resize((224, 224), antialias=True)])(img)

# Define a function to binarize a tensor image using a threshold
def binarize_image(tensor_image, threshold=190):
    grayscale_image = tensor_image.mean(dim=0, keepdim=True)
    grayscale_image = (grayscale_image * 255).numpy()

    # Apply morphological closing operation to remove noise and improve object boundaries
    kernel = np.ones((5, 5), np.uint8)
    closed_image = cv2.morphologyEx(grayscale_image, cv2.MORPH_CLOSE, kernel)

    binarized_image = (closed_image > threshold).astype(np.uint8)
    return torch.from_numpy(binarized_image).float()

# Define a custom dataset class that inherits from PyTorch's Dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, folder1, folder2):
        self.folder1 = folder1
        self.folder2 = folder2
        self.filenames = self.get_filenames(folder1)

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, index):
        filename = self.filenames[index]
        img1_path = os.path.join(self.folder1, filename)
        img2_path = os.path.join(self.folder2, filename)
        img1 = load_image(img1_path)
        img2 = load_image(img2_path)
        img2_binarized = binarize_image(img2)
        return (img1, img2_binarized)

    def get_filenames(self, folder):
        filenames = []
        for root, dirs, files in os.walk(folder):
            for file in files:
                if os.path.isfile(os.path.join(root, file)):
                    filenames.append(os.path.relpath(os.path.join(root, file), folder))
        return filenames

# Define paths to various folders
leaves_for_modelTRAIN_folder = os.path.join(google_drive_folder, 'leaves_for_modelTRAIN')
without_folder = os.path.join(leaves_for_modelTRAIN_folder, 'Without')
with_folder = os.path.join(leaves_for_modelTRAIN_folder, 'With')
test_folder = os.path.join(leaves_for_modelTRAIN_folder, 'test')
test_without_folder = os.path.join(test_folder, 'test_without')
test_with_folder = os.path.join(test_folder, 'test_with')

# Create instances of the custom dataset for training and testing
dataset = CustomDataset(with_folder, without_folder)
dataset_test = CustomDataset(test_with_folder, test_without_folder)


Load the first ten samples from the training dataset, where each sample consists of an image and its corresponding mask and displays these images and masks using the show function.

In [8]:
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import numpy as np

# Define a function to display a grid of images
def show(images, cols=10):
    cols = min(cols, len(images))
    img_grid = make_grid(images[:cols], padding=10, nrow=cols)
    plt.figure(figsize=(2 * cols, 2))
    plt.axis("off")
    plt.imshow(np.transpose(img_grid.numpy(), (1, 2, 0)))

# Create data loaders for the training and test datasets
train_loader = DataLoader(dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(dataset_test, batch_size=8, shuffle=False)

# Initialize empty lists to store images and masks
images = []
masks = []

# Iterate through the first ten samples in the training dataset
for i in range(10):
    # Get a pair of image and mask from the dataset
    image, mask = dataset[i]
    images.append(image)
    masks.append(mask)

# Display the first ten images and their masks
print("First ten images of the train dataset and their masks (224x224 rescaled)")
show(images)
show(masks)


Install the segmentation-models-pytorch package
Check if a GPU (CUDA) is available, and if so, set the device to CUDA; otherwise, set it to CPU.

In [ ]:
# Install the segmentation-models-pytorch package
!pip install segmentation-models-pytorch

# Clear the IPython output
from IPython.display import clear_output
clear_output()

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Check if GPU is available, set the device accordingly

Load fpom smp library segmentation model with the U-Net++ architecture.

Define the loss function criterion as a combination of Dice Loss and Focal Loss. Define the optimizer.

In [12]:
import segmentation_models_pytorch as smp
import torch.optim as optim

# Create a segmentation model using the U-Net++ architecture with an EfficientNet-B0 encoder
# It's a binary segmentation model with a sigmoid activation function
model = smp.UnetPlusPlus(encoder_name="efficientnet-b0", classes=1, activation="sigmoid")

# Define the loss function as a combination of DiceLoss and FocalLoss
# The loss function is weighted with 0.1 for DiceLoss and 0.9 for FocalLoss
dice_loss = smp.losses.DiceLoss(mode='binary')
focal_loss = smp.losses.FocalLoss('binary')
criterion = lambda x, y: 0.1 * dice_loss(x, y) + 0.9 * focal_loss(x, y)

# Move the model to the selected device (CPU or GPU)
model.to(device)

# Define the optimizer as AdamW with a learning rate of 0.0001 and weight decay of 1e-4
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=1e-4)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:00<00:00, 182MB/s]


Define the train function that takes the model, loss criterion, optimizer, and optionally an epoch number and a list for storing predicted masks. It performs training and validation loops for a single epoch, calculates losses, and updates the model's weights. Additionally, it prints and logs training and validation loss for the current epoch.

In [13]:
# Define a function for training the segmentation model
def train(model, criterion, optimizer, epoch=None, masks_in_progress=[]):
    ep_loss = 0  # Initialize the epoch loss to 0
    model.train()  # Set the model to training mode
    for img_batch, masks_batch in train_loader:  # Iterate through training data batches
        optimizer.zero_grad()  # Zero out the gradients
        output = model(img_batch.to(device))  # Forward pass: compute model predictions
        loss = criterion(output, masks_batch.to(device))  # Calculate the loss
        loss.backward()  # Backpropagation: compute gradients
        optimizer.step()  # Update model weights using the optimizer
        ep_loss += loss.item()  # Accumulate the epoch loss

    val_loss = 0  # Initialize the validation loss to 0
    for i, batch in enumerate(test_loader):  # Iterate through validation data batches
        with torch.no_grad():  # Disable gradient computation for validation
            img_batch, masks_batch = batch
            output = model(img_batch.to(device))  # Forward pass for validation data
            loss = criterion(output, masks_batch.to(device))  # Calculate validation loss
            val_loss += loss.item()  # Accumulate the validation loss
            if i == 0:
                masks_in_progress.append(output[1].cpu())  # Store predicted masks for visualization

    print(
        "Epoch {} Train loss {:.2f} Val loss {:.2f}".format(
            epoch, ep_loss / len(train_loader), val_loss / len(test_loader)
        )
    )  # Print and log training and validation loss for the current epoch


Train the model for 120 epochs

In [ ]:
%%time  # Measure the time taken for training
masks_in_progress = []  # Initialize a list to store predicted masks during training

# Iterate through a specified number of epochs (120 in this case)
for epoch in range(120):
    # Call the 'train' function to train the model for one epoch
    train(model, criterion, optimizer, epoch, masks_in_progress)

# Save the trained model's state dictionary to a file
torch.save(model.state_dict(), os.path.join(google_drive_folder, "trained_model"))

# Print a message indicating the completion of training and the location of the saved model
print(f"Training of the model is complete. The trained model is located in the folder {google_drive_folder}")


Display the results of the neural network on images with artificially damaged leaves

In [ ]:
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import numpy as np

# Define a function to display a grid of images
def show(batch, cols=16, max_size=256):
    cols = min(cols, len(batch))
    # Create a grid of images with specified padding, normalization, and scaling
    img_grid = make_grid(batch[:cols], padding=10, nrow=cols, normalize=True, scale_each=True, max_size=max_size)
    plt.figure(figsize=(cols, cols))
    plt.axis("off")
    plt.imshow(np.transpose(img_grid.cpu().numpy(), (1, 2, 0)))

# Define a function to display model predictions on the validation dataset
def show_valset_pred(model, cols=16):
    images, pred_masks = [], []
    for batch in dataset_test:  # Iterate through the validation dataset
        with torch.no_grad():
            img, mask = batch
            images.append(img.unsqueeze(0))  # Append the original image
            output = model(img.unsqueeze(0).to(device))  # Forward pass for prediction
            pred_masks.append(output.cpu())  # Append the predicted mask
    # Display the original images, predicted masks, and the difference between them
    show(torch.stack(images).squeeze()[:cols, ...])
    show(torch.stack(pred_masks).squeeze(1)[:cols, ...])
    show(torch.stack(pred_masks).squeeze(1)[:cols, ...] - torch.stack(images).squeeze()[:cols, ...])

# Display the results of the neural network on images with artificially damaged leaves
print("Neural network results on images with artificially damaged leaves")
show_valset_pred(model)
